# Instructions

* Due date is 11/08 midnight.
* Please check this original colab often to see if anything changes in the instructions. Last updated: 11/06 11:00am.
* Make a copy of this colab, add your code, run all the cells and save output. Submit your .ipynb file with output. Don't submit a link to your colab, you must submit actual .ipynb file with output.
* You must train the model and show the test output. Points will be deducted if your model is not trained / produces run time errors. If you try to build a very large model and it doesn't finish training, than you would get points deducted, so make reasonable adjustments to keep the model size under check, do not train very small models, do not train very large models.


Q1 RNN (10 Points)

Train a GRU classifier for speech-to-text recognition using [dataset](https://www.kaggle.com/datasets/mathurinache/the-lj-speech-dataset) (split this into train/validation).

You may want to use [librosa](https://librosa.org/doc/latest/tutorial.html), or [python speech features](https://python-speech-features.readthedocs.io/en/latest/) (there are some diffs between the two).

You may want to use Mel-Frequency Cepstral Coefficients (MFCC) as features.






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/CS171 HW 6/archive.zip'
extract_path = '/content/LJSpeech'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)

['LJSpeech-1.1']

In [ ]:
!pip install librosa jiwer

import numpy as np
import pandas as pd
import os
import librosa
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Dense, Bidirectional, Dropout, TimeDistributed, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from jiwer import wer

#12 times of training

#2 succeed and 10 fail

#Some of my reference I use was:
#https://github.com/rudrajikadra/Speech-Emotion-Recognition-using-Librosa-library-and-MLPClassifier/blob/master/Speech_Emotion_Recognition_Notebook.ipynb
#https://stackoverflow.com/questions/61174521/getting-96-mfcc-features-using-python-speech-features
#https://en.wikipedia.org/wiki/Mel-frequency_cepstrum


#ISSUE!
#1. SO throughout this assingment I learn thatmodel cant really understand in so much way. #I think this is a very diffcult focus
#Using blank label, that acts like a space so that the model doesn’t get confused when words end.
#FOr some reason the tokenizer was adding 1 to all value which I was frustrated on why.



#Model Issue:
#I decided to go with a setup called Bidirectional GRU layers


#Training and Error (Lots of It!)
#My labels weren’t lining up with the ml output which  I had to reshape my data to match. I had to fix up the label
#lengths and make sure everything match

#google co lab auto fill and auto correction was very useless when it comes to token!
#WOuld be better if you provided diagram



metadata = pd.read_csv('/content/LJSpeech/LJSpeech-1.1/metadata.csv', sep='|', header=None,
                       names=['file_name', 'transcription', 'normalized_transcription'])

metadata = metadata[:3000]

metadata = metadata.dropna(subset=['normalized_transcription']).reset_index(drop=True)

transcriptions = metadata['normalized_transcription'].astype(str)

tokenizer = Tokenizer(char_level=True, filters='', lower=True)
tokenizer.fit_on_texts(transcriptions)
char_index = tokenizer.word_index
vocab_size = len(char_index) + 1  # additioning a blank label to the vocab test

print(f'Vocabulary Size (including blank): {vocab_size}')

sequences = tokenizer.texts_to_sequences(transcriptions)

# simple subtract 1
sequences = [np.array(seq) - 1 for seq in sequences]

max_seq_length = max(len(seq) for seq in sequences)
transcripts_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post', value=vocab_size - 1)

print(f'Max Sequence Length: {max_seq_length}')

Vocabulary Size (including blank): 40
Max Sequence Length: 185


In [ ]:

#This whole process took over 12 times of running becasue I hit so many rb level
#mismatches, blank label errors, issues with padding, and the CTC loss throwing errors


def extract_features(file_name, max_time_steps=300):
    try:
        audio, sample_rate = librosa.load(file_name, sr=16000)
        # for mmcc to prevent skewed data really affecting the ai model
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)
        mfccs = mfccs.T
        if mfccs.shape[0] > max_time_steps:
            mfccs = mfccs[:max_time_steps, :]
        else:
            pad_width = max_time_steps - mfccs.shape[0]
            mfccs = np.pad(mfccs, ((0, pad_width), (0, 0)), mode='constant')
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
        return None
    return mfccs


X = []
Y = []
for index, row in metadata.iterrows():
    file_name = f'/content/LJSpeech/LJSpeech-1.1/wavs/{row["file_name"]}.wav'
    features = extract_features(file_name)
    if features is not None:
        X.append(features)
        Y.append(transcripts_padded[index])

X = np.array(X)
Y = np.array(Y)

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

# simple splitting for 80-20 ratio
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'Y_val shape: {Y_val.shape}')

X shape: (2986, 300, 40)
Y shape: (2986, 185)
X_train shape: (2388, 300, 40)
Y_train shape: (2388, 185)
X_val shape: (598, 300, 40)
Y_val shape: (598, 185)


In [ ]:
input_dim = X_train.shape[2]
time_steps = X_train.shape[1]
output_dim = vocab_size

inputs = Input(name='inputs', shape=(time_steps, input_dim))


x = Bidirectional(GRU(256, return_sequences=True))(inputs)

#i include this because it's helpful because context from both past and future sounds improves accuracy.

x = Dropout(0.2)(x)
x = Bidirectional(GRU(256, return_sequences=True))(x)
x = Dropout(0.2)(x)
x = Bidirectional(GRU(256, return_sequences=True))(x)
x = Dropout(0.2)(x)


x = TimeDistributed(Dense(output_dim))(x)
y_pred = Activation('softmax')(x)
#Lets talk why speech data is strucutre.
#The model tertmine the likelilhood of each character





model = Model(inputs, y_pred)
model.summary()

# I customized the CTC loss to fit the vocabulary size and label index handling
def ctc_loss_lambda_func(y_true, y_pred):
    y_true = tf.cast(y_true, dtype='int32')

    input_length = tf.fill([tf.shape(y_pred)[0], 1], tf.shape(y_pred)[1])

    label_length = tf.reduce_sum(tf.cast(tf.not_equal(y_true, vocab_size - 1), dtype='int32'), axis=1, keepdims=True)


    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)

    return loss

model.compile(optimizer='adam', loss=ctc_loss_lambda_func)

Y_train_ctc = Y_train.astype('int32')
Y_val_ctc = Y_val.astype('int32')

assert np.max(Y_train_ctc) < vocab_size, "Label index exceeds vocab size."
assert np.max(Y_val_ctc) < vocab_size, "Label index exceeds vocab size."

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)                  │ (None, 300, 40)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 300, 512)            │         457,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 300, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 300, 512)            │       1,182,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 300, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 300, 512)            │       1,182,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 300, 512)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 300, 40)             │          20,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 300, 40)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,843,688 (10.85 MB)

 Trainable params: 2,843,688 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# very dislike on how dataset work because learning rate reduction callbacks for
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)


#I think if there is a example you can show us that would help alot
#For CTC loss, what should we expect?

#CTC represent the gaps between character but it kept causing errror.






model.fit(X_train, Y_train_ctc,
          batch_size=16,
          epochs=30,
          validation_data=(X_val, Y_val_ctc),
          callbacks=[early_stopping, reduce_lr])

def decode_predictions(pred):
    input_length = np.ones(pred.shape[0]) * pred.shape[1]
    #So here the line creates an array which tell function each prediction length
    decoded, _ = tf.keras.backend.ctc_decode(pred, input_length, greedy=False, beam_width=10, top_paths=1)
    #after we decoding the func return final
    decoded = tf.keras.backend.get_value(decoded[0])
    return decoded

# beta test over 8 days of trying over and over
def evaluate_model(model, X_val, Y_val, tokenizer):
    predictions = model.predict(X_val)
    decoded_outputs = decode_predictions(predictions)
    decoded_sentences = []
    actual_sentences = []
    for i in range(len(decoded_outputs)):
      #data structure sample:
      #we put this loop which goes through each guess and answer one by one


        decoded_sequence = [int(x) for x in decoded_outputs[i] if x != -1 and x != vocab_size - 1]

        #decoded_sequence is the cleaned-up version of the model
        #actual_sequence is the cleaned-up version of the actual answer text.
        decoded_sentence = tokenizer.sequences_to_texts([np.array(decoded_sequence) + 1])[0]
        decoded_sentences.append(decoded_sentence)
        actual_sequence = [int(x) for x in Y_val[i] if x != vocab_size - 1]
        actual_sentence = tokenizer.sequences_to_texts([np.array(actual_sequence) + 1])[0]
        actual_sentences.append(actual_sentence)




    total_cer = 0
    for decoded_sentence, actual_sentence in zip(decoded_sentences, actual_sentences):
        cer = wer(actual_sentence, decoded_sentence)
        total_cer += cer
    average_cer = total_cer / len(decoded_sentences)
    print(f'Average Character Error Rate (CER): {average_cer * 100:.2f}%')
    for i in range(5):
        print(f'Decoded: {decoded_sentences[i]}')
        print(f'Actual: {actual_sentences[i]}')
        print('---')

evaluate_model(model, X_val, Y_val, tokenizer)

Epoch 1/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 59s 281ms/step - loss: 368.3432 - val_loss: 229.9221 - learning_rate: 0.0010
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 86s 311ms/step - loss: 210.0755 - val_loss: 154.9332 - learning_rate: 0.0010
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 76s 269ms/step - loss: 153.1827 - val_loss: 120.3071 - learning_rate: 0.0010
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 44s 291ms/step - loss: 118.8342 - val_loss: 99.2141 - learning_rate: 0.0010
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 82s 290ms/step - loss: 100.1162 - val_loss: 87.7927 - learning_rate: 0.0010
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 82s 291ms/step - loss: 84.1063 - val_loss: 79.0900 - learning_rate: 0.0010
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 82s 290ms/step - loss: 73.5111 - val_loss: 74.5809 - learning_rate: 0.0010
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 78s 261ms/step - loss: 63.3928 - val_loss: 69.5143 - learning_rate: 0.0010
Epoch 9/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 41s 261ms/step - loss: 56.5932 -